In [1]:
import os
if "models" not in os.listdir("."):
    os.chdir("../..")

In [2]:
%load_ext autoreload
%autoreload 2
import penzai
import jax_smi
jax_smi.initialise_tracking()
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [3]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/gemma-2-2b-it.gguf",
                                         from_type="gemma2",
                                         load_eager=True
                                         )

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("alpindale/gemma-2b")
tokenizer.padding_side = "right"

In [5]:
from sprint.icl_sfc_utils_gemma_2 import Circuitizer

In [6]:
from sprint.task_vector_utils import load_tasks, ICLRunner
tasks = load_tasks()

fatal: destination path 'data/itv' already exists and is not an empty directory.


In [7]:
task_names = ["en_es", "antonyms", "person_profession"]
task_name = task_names[0]
task_name = "antonyms"

In [8]:
%env JAX_TRACEBACK_FILTERING=off
import jax
jax.config.update('jax_traceback_filtering', 'off')

env: JAX_TRACEBACK_FILTERING=off


In [9]:
def check_if_single_token(token):
    return len(tokenizer.tokenize(token)) == 1

task = tasks[task_name]

print(len(task))

# task = {
#     k:v for k,v in task.items() if check_if_single_token(k) and check_if_single_token(v)
# }

print(len(task))

pairs = list(task.items())

batch_size = 8 
n_shot=16
if task_name.startswith("algo"):
    n_shot = 8
max_seq_len = 128
seed = 10

prompt = "Follow the pattern:\n{}"

runner = ICLRunner(task_name, pairs, batch_size=batch_size, n_shot=n_shot, max_seq_len=max_seq_len, seed=seed, prompt=prompt, use_same_examples=False, use_same_target=False)


layers = list(range(15, 22))
circuitizer = Circuitizer(llama, tokenizer, runner, layers, prompt)

# circuitizer.run_metrics()

162
162
Setting up masks...


Running metrics...
Setting up RMS...


  0%|          | 0/26 [00:00<?, ?it/s]

Loading SAEs...


  0%|          | 0/7 [00:00<?, ?it/s]

Running node IEs...


  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [11]:
import numpy as np
# thresholds = np.linspace(-1e-2, 1e-4, 200)
# thresholds = np.linspace(1.4 * 1e-4, 1.45 * 1e-4, 200)
# thresholds = np.logspace(-7, -2, 100)
thresholds = np.linspace(0, 1e-5, 50)
topks = [4, 6, 12, 16, 24, 32]


ablated_metrics, n_nodes_counts = circuitizer.run_ablated_metrics(thresholds, inverse=False, 
                                                                  do_abs=True, mean_ablate=False, 
                                                                  average_over_positions=True,
                                                                  token_prefix=None)



# target_metric = (max(ablated_metrics) - min(ablated_metrics)) * 0.95 + min(ablated_metrics)
# target_threshold = [threshold for threshold, metric in reversed(list(zip(thresholds, ablated_metrics))) if metric > target_metric][0]

  0%|          | 0/50 [00:00<?, ?it/s]

In [23]:
circuitizer.run_ablated_metrics([-1, 0, 1e-7], sae_types=["transcoder", "attn_out"], do_abs=True, average_over_positions=True)

  0%|          | 0/3 [00:00<?, ?it/s]

([-0.12146639823913574, -0.12444095313549042, -0.11790468543767929],
 [1966080, 45566, 37332])

In [32]:
circuitizer.ablated_metric(llama)

<jax.Array(-0.119629, dtype=bfloat16)>

In [14]:
import matplotlib.pyplot as plt
import plotly.express as px

# plt.plot([max(n_nodes_counts) - x for x in n_nodes_counts], ablated_metrics)
# plt.plot(thresholds, ablated_metrics)
# plt.plot(thresholds, ablated_metrics)
# plt.plot(n_nodes_counts, ablated_metrics)
# plt.plot(thresholds, n_nodes_counts)
# plt.xscale("log")
# plt.plot(n_nodes_counts)

# px.line(x=list(range(len(ablated_metrics))), y=ablated_metrics)
# fig
fig = px.line(x=thresholds, y=ablated_metrics)
# fig.update_xaxes(type="log", exponentformat="power")
fig

# px.line(x=n_nodes_counts, y=ablated_metrics)


In [35]:
import json
with open("gemma_2_cleaning_compact_65k.jsonl") as f:
    data = [json.loads(line) for line in f]

In [36]:
task_data = [d for d in data if d["task"] == task_name and d["layer"] == 20]

In [37]:
task_data

[{'task': 'antonyms',
  'weights': [68.01600646972656,
   47.43989562988281,
   34.77153396606445,
   22.7160587310791,
   12.102900505065918],
  'indices': [17969, 35629, 8113, 51920, 22490],
  'tv loss': 3.078125,
  'cleaning loss': 3.609375,
  'ito loss': 3.65625,
  'layer': 20}]

In [18]:
import jax.numpy as jnp

tokens = circuitizer.train_tokens
mask = tokens[:, 1:] == 3978
mask = jnp.cumsum(mask[:, ::-1], axis=-1)[:, ::-1] > 0
mask = jnp.logical_not(mask)

mask = jnp.logical_and(mask, tokens[:, 1:] != 0)


tokens[0, 1:][mask[0]]

array([13488])

In [ ]:
runner.train_pairs

In [32]:
circuitizer.ie_resid[18][0, [60, 64, 68], 44111]

<jax.Array([-0.00028572, -0.00039319, -0.01139488], dtype=float32)>

In [10]:
from micrlhf.llama import LlamaBlock, LlamaAttention, LlamaMLP

# llama.select().at_instances_of(LlamaBlock).pick_nth_selected(18).get()

In [11]:
get_resids = llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
            pz.nn.Sequential([
                x,
                pz.de.TellIntermediate.from_config(tag=f"resid_pre_{i}"),
            ])
        )

get_resids = get_resids.select().at_instances_of(LlamaAttention).apply_with_selected_index(lambda l, b: b.select().at(lambda x: x.attn_value_to_output).at_instances_of(pz.nn.Linear).apply_with_selected_index(lambda i, x: pz.nn.Sequential([
            pz.de.TellIntermediate.from_config(tag=f"resid_attn_{l}"),
            x,
        ])))

get_resids = get_resids.select().at_instances_of(LlamaMLP).apply_with_selected_index(lambda i, x:
    pz.nn.Sequential([
        pz.de.TellIntermediate.from_config(tag=f"resid_mlp_in_{i}"),
        x,
        pz.de.TellIntermediate.from_config(tag=f"resid_mlp_out_{i}"),
    ])
)

get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: True)

In [12]:
# get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: True)

In [12]:
tokens = runner.get_tokens(
    runner.train_pairs, tokenizer
)["input_ids"]

tokens_wrapped = pz.nx.wrap(tokens, "batch", "seq")
inputs = llama.inputs.from_basic_segments(tokens_wrapped)

logits, resids = get_resids(inputs)

In [15]:
[x for x in resids[:5]]

[SideOutputValue(
   keypath=(GetAttrKey(name='body'), GetAttrKey(name='body'), GetAttrKey(name='sublayers'), SequenceKey(idx=3), GetAttrKey(name='sublayers'), SequenceKey(idx=0), GetAttrKey(name='sublayers'), SequenceKey(idx=0), GetAttrKey(name='sublayers'), SequenceKey(idx=1), GetAttrKey(name='delta'), GetAttrKey(name='sublayers'), SequenceKey(idx=2), GetAttrKey(name='attn_value_to_output'), GetAttrKey(name='sublayers'), SequenceKey(idx=1), GetAttrKey(name='sublayers'), SequenceKey(idx=0), GetAttrKey(name='side_out')),
   tag='resid_attn_0',
   value=<NamedArray bfloat16(| batch:8, seq:128, kv_heads:4, q_rep:2, projection:256) ≈-0.0014 ±0.34 [≥-4.2, ≤5.4] nonzero:2_097_152 (wrapping jax.Array)>,
 ),
 SideOutputValue(
   keypath=(GetAttrKey(name='body'), GetAttrKey(name='body'), GetAttrKey(name='sublayers'), SequenceKey(idx=3), GetAttrKey(name='sublayers'), SequenceKey(idx=0), GetAttrKey(name='sublayers'), SequenceKey(idx=0), GetAttrKey(name='sublayers'), SequenceKey(idx=3), GetAttrKey(name='delta'), GetAttrKey(name='sublayers'), SequenceKey(idx=2), GetAttrKey(name='sublayers'), SequenceKey(idx=0), GetAttrKey(name='side_out')),
   tag='resid_mlp_in_0',
   value=<NamedArray bfloat16(| batch:8, seq:128, embedding:2304) ≈0.0052 ±1.9 [≥-4e+01, ≤7.8e+01] zero:1_935 nonzero:2_357_361 (wrapping jax.Array)>,
 ),
 SideOutputValue(
   keypath=(GetAttrKey(name='body'), GetAttrKey(name='body'), GetAttrKey(name='sublayers'), SequenceKey(idx=3), GetAttrKey(name='sublayers'), SequenceKey(idx=0), GetAttrKey(name='sublayers'), SequenceKey(idx=0), GetAttrKey(name='sublayers'), SequenceKey(idx=3), GetAttrKey(name='delta'), GetAttrKey(name='sublayers'), SequenceKey(idx=2), GetAttrKey(name='sublayers'), SequenceKey(idx=2), GetAttrKey(name='side_out')),
   tag='resid_mlp_out_0',
   value=<NamedArray bfloat16(| batch:8, seq:128, embedding:2304) ≈-0.0015 ±0.18 [≥-1.1e+01, ≤5.6] nonzero:2_359_296 (wrapping jax.Array)>,
 ),
 SideOutputValue(
   keypath=(
     GetAttrKey(name='body'),
     GetAttrKey(name='body'),
     GetAttrKey(name='sublayers'),
     SequenceKey(idx=3),
     GetAttrKey(name='sublayers'),
     SequenceKey(idx=0),
     GetAttrKey(name='sublayers'),
     SequenceKey(idx=1),
     GetAttrKey(name='side_out'),
   ),
   tag='resid_pre_0',
   value=<NamedArray bfloat16(| batch:8, seq:128, embedding:2304) ≈-0.0068 ±2.0 [≥-1e+02, ≤3.3e+02] zero:2_091 nonzero:2_357_205 (wrapping jax.Array)>,
 ),
 SideOutputValue(
   keypath=(GetAttrKey(name='body'), GetAttrKey(name='body'), GetAttrKey(name='sublayers'), SequenceKey(idx=3), GetAttrKey(name='sublayers'), SequenceKey(idx=1), GetAttrKey(name='sublayers'), SequenceKey(idx=0), GetAttrKey(name='sublayers'), SequenceKey(idx=1), GetAttrKey(name='delta'), GetAttrKey(name='sublayers'), SequenceKey(idx=2), GetAttrKey(name='attn_value_to_output'), GetAttrKey(name='sublayers'), SequenceKey(idx=1), GetAttrKey(name='sublayers'), SequenceKey(idx=0), GetAttrKey(name='side_out')),
   tag='resid_attn_1',
   value=<NamedArray bfloat16(| batch:8, seq:128, kv_heads:4, q_rep:2, projection:256) ≈0.0049 ±0.31 [≥-4.2, ≤4.1] nonzero:2_097_152 (wrapping jax.Array)>,
 )]